In [6]:
!pip install langchain tiktoken langchain_experimental sentence-transformers google-generativeai

In [25]:
from langchain.llms import GooglePalm

import yaml
with open("config.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)


api_key = cfg['creds']['api_creds']['api_key']
llm = GooglePalm(google_api_key=api_key, temperature=0.1)


In [16]:
llm("write a short summary of what Google Palm can do with respect to SQL code generation")

/Users/archanganguly/Documents/2023/PMI/GenAI/PoCs/AutoGPT/.conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Google Palm can generate SQL code for data transformations, including joins, aggregations, and filtering. It can also generate schema and table definitions.'

In [28]:
from langchain.utilities import SQLDatabase

db_user = cfg['creds']['db_creds']['username']
db_password = cfg['creds']['db_creds']['password']
db_host = cfg['creds']['db_creds']['host']
db_name = cfg['creds']['db_creds']['database']

# schema to access is "logistics". Replace schema name here with your database schema

db = SQLDatabase.from_uri(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}", schema="logistics", sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE logistics.commodity (
	commoditycode VARCHAR(20) NOT NULL, 
	commoditydescription TEXT, 
	customersku INTEGER, 
	vendorsku INTEGER, 
	grossweight REAL, 
	uom VARCHAR(5), 
	validfrom DATE, 
	validto DATE, 
	sellingprice REAL, 
	currency VARCHAR(10)
)

/*
3 rows from commodity table:
commoditycode	commoditydescription	customersku	vendorsku	grossweight	uom	validfrom	validto	sellingprice	currency
COKE250	Coca Cola 250ml	11001	91001	250.0	ml	None	None	1.0	USD
COKE500	Coca Cola 500ml	11002	91002	500.0	ml	None	None	1.75	USD
COKE2000	Coca Cola 2L	11003	91003	2000.0	L	None	None	1.75	USD
*/


CREATE TABLE logistics.partyaddressdetails (
	partyaddressid INTEGER GENERATED ALWAYS AS IDENTITY (INCREMENT BY 1 START WITH 1 MINVALUE 1 MAXVALUE 2147483647 CACHE 1 NO CYCLE), 
	addressline1 VARCHAR(200), 
	addressline2 VARCHAR(200), 
	addressline3 VARCHAR(200), 
	addressline4 VARCHAR(200), 
	postalcode VARCHAR(20), 
	citycode VARCHAR(20), 
	provincecode VARCHAR(100), 
	countrycode VARCHAR(10

In [29]:
!pip install psycopg2-binary

In [30]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qnst1 = db_chain("What is the total gross weight of orders that are fulfilled?")

/Users/archanganguly/Documents/2023/PMI/GenAI/PoCs/AutoGPT/.conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
What is the total gross weight of orders that are fulfilled?
SQLQuery:SELECT SUM(orderedtotalgrossweight) FROM serviceorder WHERE orderstatus = 'Fulfilled'
SQLResult: [(5874.0,)]
Answer:5874.0
> Finished chain.


In [15]:
qnst1

{'query': 'What is the total gross weight of orders that are fulfilled?',
 'result': '5874.0'}

In [34]:
qnst2 = db_chain.run("How many Chocolate based orders are fulfilled till now ?")



> Entering new SQLDatabaseChain chain...
How many Chocolate based orders are fulfilled till now ?
SQLQuery:SELECT COUNT(*) FROM serviceorderline WHERE orderedproductcode = 'CHOCOLATE' AND orderstatus = 'Fulfilled'
SQLResult: [(11,)]
Answer:11
> Finished chain.


In [173]:
qnst2

'21'

In [15]:
qnst3 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")

/Users/archanganguly/Documents/2023/PMI/GenAI/PoCs/AutoGPT/.conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) FROM inventory.t_shirts WHERE size = 'S'
SQLResult: [(76,)]
Answer:76
> Finished chain.
